In [1]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install numpy scipy sklearn
!pip install jax
!pip install chex
!pip install attr
!pip install optax



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from Sinkhorn_CMI import SinkhornImputation_CMI
from RR_imputer import RRimputer
import matplotlib.pyplot as plt
from CMI import *

from Inject_Missing_Values import *

from Miss import *

import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [3]:
import numpy as np
#from mbi import Domain, CliqueVector, Factor, LinearMeasurement
#from mbi import marginal_oracles, marginal_loss, synthetic_data
from typing import Callable
import jax
import jax.numpy as jnp
import chex
import attr
import optax

In [4]:
import pandas as pd

# Read CSV file into a Pandas DataFrame
groundTruth = pd.read_csv(r'C:\Users\zhossai3\Desktop\My code\OT_Fairness\Data\German Credit\German_Credit_Ground_Truth.csv', delimiter=',', header=0)

# Store feature columns in a DataFrame

X= groundTruth.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = groundTruth.iloc[:, -1]  # Selects all rows and only the last column

# Display the DataFrames
print(X)  # Print first few rows of features
 # Print first few rows of target variable
type(X)
print(type(Y))


     checking-account  duration  credit-history  purpose  credit-amount  \
0                   1         6               0        7           1169   
1                   0        48               1        7           5951   
2                   3        12               0        4           2096   
3                   1        42               1        5           7882   
4                   1        24               2        1           4870   
..                ...       ...             ...      ...            ...   
995                 3        12               1        5           1736   
996                 1        30               1        2           3857   
997                 3        12               1        7            804   
998                 1        45               1        7           1845   
999                 0        45               0        2           4576   

     savings-account  employment-since  installment-rate  other-debtors  \
0                  4    

In [5]:
groundTruth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   checking-account                       1000 non-null   int64
 1   duration                               1000 non-null   int64
 2   credit-history                         1000 non-null   int64
 3   purpose                                1000 non-null   int64
 4   credit-amount                          1000 non-null   int64
 5   savings-account                        1000 non-null   int64
 6   employment-since                       1000 non-null   int64
 7   installment-rate                       1000 non-null   int64
 8   other-debtors                          1000 non-null   int64
 9   residence-since                        1000 non-null   int64
 10  property                               1000 non-null   int64
 11  age                            

In [6]:
groundTruth['job'].unique()

array([0, 1, 2, 3])

In [7]:
def conditional_mutual_information(data, X, Y, Z, delta=1):

  cmi = 0

  P_Z = data.groupby(Z).size()
  P_Z = P_Z / P_Z.sum()

  P_XZ = data.groupby(X + Z).size()
  P_XZ = P_XZ / P_XZ.sum()

  P_YZ = data.groupby(Y + Z).size()
  P_YZ = P_YZ / P_YZ.sum()

  P_XYZ = data.groupby(X + Y + Z).size()
  P_XYZ = P_XYZ / P_XYZ.sum()

  for ind in P_XYZ.index:
    x_ind = ind[:len(X)]
    y_ind = ind[len(X):len(X + Y)]
    z_ind = ind[len(X + Y):]

    xz_ind = x_ind + z_ind
    yz_ind = y_ind + z_ind
    xyz_ind = ind

    z_ind = pd.MultiIndex.from_tuples([z_ind], names=Z) if len(Z) != 1 else pd.Index(z_ind, name=Z[0])
    xz_ind = pd.MultiIndex.from_tuples([xz_ind], names=X + Z)
    yz_ind = pd.MultiIndex.from_tuples([yz_ind], names=Y + Z)
    xyz_ind = pd.MultiIndex.from_tuples([xyz_ind], names=X + Y + Z)

    cmi += delta * P_XYZ[xyz_ind].item() * np.log2(
      P_Z[z_ind].item() * P_XYZ[xyz_ind].item() / (P_XZ[xz_ind].item() * P_YZ[yz_ind].item()))

  return cmi

In [8]:
data = groundTruth # Convert tensor to numpy array
sensitive_attributes = [11, 19]

# Define different bin sizes for each sensitive attribute
bin_sizes = {
    11: 4,  # Binary attribute
    19: 2  # More categories
}


# Digitize multiple sensitive attributes with different bin sizes
#X_list = [
 #   np.digitize(data[:, col], bins=np.linspace(data[:, col].min(), data[:, col].max(), bin_sizes[col] + 1))
  #  for col in sensitive_attributes
#]
#x = np.column_stack(X_list)
#X = np.digitize(data[:,1],bins=np.linspace(data[:,1].min(), data[:,1].max(),1)) 
#y =np.digitize(Y,bins=np.linspace(Y.min(), Y.max(),4)) 
#z =  np.digitize(data[:, 7],bins=np.linspace(data[:, 7].min(), data[:, 7].max(), 4))

x = ['sex','age']
y = ['class-label']
z = ['credit-amount', 'job']


cmi_value =conditional_mutual_information(data, x, y, z, delta=1)
print("Conditional Mutual Information:", cmi_value)

Conditional Mutual Information: 0.027509775004326952


**30% MCAR**

In [9]:
generator = Inject_Missing_Values()

In [10]:
type(Y)

pandas.core.series.Series

In [11]:
miss_mcar30,index_mcar30 = generator.MCAR(X,missing_rate=30)
type(miss_mcar30)
miss_mcar30 = pd.concat([miss_mcar30, Y], axis=1)

In [12]:
miss_mcar30_numpy = scale(miss_mcar30)
X_numpy = scale(X)
Y_numpy = scale(Y)

miss_mcar30_tensor = torch.tensor(miss_mcar30_numpy)
groundTruth_tensor = torch.tensor(scale(groundTruth))
X_tensor = torch.tensor(X_numpy)
Y_tensor = torch.tensor(Y_numpy)

In [13]:
#sinkhorn
n_mcar30, d_mcar30 = miss_mcar30_tensor.shape
batchsize = 128 # If the batch size is larger than half the dataset's size,
                # it will be redefined in the imputation methods.
lr = 1e-2
epsilon_mcar30 = pick_epsilon(miss_mcar30_tensor)
#epsilon_mcar30 = 0.3
print(epsilon_mcar30)


sk_imputer_mcar30 = SinkhornImputation(eps=epsilon_mcar30, batchsize=batchsize, lr=lr, niter=2000)
sk_imp_mcar30, sk_maes30mcar, sk_rmses30mcar = sk_imputer_mcar30.fit_transform(miss_mcar30_tensor, verbose=True, report_interval=50, X_true=groundTruth_tensor)
#using numpy version of data
#sk_imp_mcar30_numpy = sk_imp_mcar30.detach().cpu().numpy()

#sk_mae_mcar30 = MAE(sk_imp_mcar30_numpy,ground_truth_numpy ,mask_mcar30)
#sk_rmse_mcar30 = RMSE(sk_imp_mcar30_numpy, ground_truth_numpy,mask_mcar30)
#print(sk_mae_mcar30,sk_rmse_mcar30)

0.7173755433480349


INFO:root:batchsize = 128, epsilon = 0.7174
INFO:root:Iteration 0:	 Loss: 5.6585	 Validation MAE: 0.7737	RMSE: 1.0038
INFO:root:Iteration 50:	 Loss: 4.8395	 Validation MAE: 0.7080	RMSE: 0.9911
INFO:root:Iteration 100:	 Loss: 5.5765	 Validation MAE: 0.6960	RMSE: 0.9859
INFO:root:Iteration 150:	 Loss: 5.2007	 Validation MAE: 0.6920	RMSE: 0.9846
INFO:root:Iteration 200:	 Loss: 5.0344	 Validation MAE: 0.6894	RMSE: 0.9844
INFO:root:Iteration 250:	 Loss: 6.0266	 Validation MAE: 0.6873	RMSE: 0.9841
INFO:root:Iteration 300:	 Loss: 5.4672	 Validation MAE: 0.6846	RMSE: 0.9833
INFO:root:Iteration 350:	 Loss: 5.0426	 Validation MAE: 0.6845	RMSE: 0.9841
INFO:root:Iteration 400:	 Loss: 5.4224	 Validation MAE: 0.6834	RMSE: 0.9848
INFO:root:Iteration 450:	 Loss: 5.1778	 Validation MAE: 0.6842	RMSE: 0.9849
INFO:root:Iteration 500:	 Loss: 5.1321	 Validation MAE: 0.6845	RMSE: 0.9854
INFO:root:Iteration 550:	 Loss: 4.9572	 Validation MAE: 0.6847	RMSE: 0.9869
INFO:root:Iteration 600:	 Loss: 5.3140	 Validat

In [14]:
print(sk_imp_mcar30)

tensor([[ 0.3786, -0.1369, -1.0055,  ...,  0.0212, -0.3271,  0.6547],
        [-0.6479,  2.1846, -0.0381,  ..., -1.4670, -0.2245, -1.5275],
        [ 1.1298, -0.7904, -1.0055,  ...,  0.5638, -0.3278,  0.6547],
        ...,
        [ 1.1298, -0.7263, -0.5070,  ...,  0.6816, -0.3237,  0.6547],
        [-0.4551,  1.9420, -0.0381,  ...,  0.6816, -0.2665, -1.5275],
        [-1.2476, -0.1763, -0.1424,  ...,  0.6816, -0.3278,  0.6547]],
       grad_fn=<IndexPutBackward0>)


In [15]:
sex = sk_imp_mcar30[:,19]
age = sk_imp_mcar30[:,11]
print(sex)

tensor([ 2.1232e-02, -1.4670e+00,  5.6381e-01,  6.8165e-01,  6.8165e-01,
         6.8165e-01,  6.8165e-01,  4.3857e-01,  6.8165e-01,  6.8165e-01,
        -1.4670e+00, -1.4670e+00,  3.6430e-01,  6.8165e-01, -1.4670e+00,
         4.2386e-01,  6.8165e-01,  6.8165e-01, -1.4670e+00,  6.8165e-01,
        -1.0031e-01,  4.1124e-01,  6.8165e-01,  6.8165e-01,  6.8165e-01,
         6.8165e-01,  6.8165e-01,  1.9200e-02,  6.8165e-01,  4.0992e-01,
         6.8165e-01,  6.8165e-01,  6.8165e-01,  6.8165e-01,  6.4858e-02,
         6.8165e-01,  6.8165e-01,  4.5954e-01,  6.8165e-01,  6.8165e-01,
         7.0465e-02,  6.8165e-01,  6.8165e-01,  6.8165e-01, -1.4670e+00,
        -1.4670e+00,  5.8717e-01, -9.5280e-01,  6.8165e-01, -1.4670e+00,
         5.0900e-01,  6.8165e-01,  6.8165e-01,  6.8165e-01,  6.8165e-01,
         2.8098e-01,  2.9899e-01, -1.4670e+00, -1.4670e+00, -1.4670e+00,
         6.8165e-01,  6.8165e-01,  6.8165e-01,  6.8165e-01, -1.4670e+00,
         4.8932e-01, -1.4670e+00,  6.8165e-01,  6.8

In [16]:
sex_bucketize = torch.bucketize(sex,  torch.linspace(sex.min(), sex.max(), steps=2))
print(sex_bucketize)
#age_bucketize = torch.bucketize(age,  torch.linspace(age.min(), age.max(), steps=10))
#print(age_bucketize)

tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
        0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,

In [17]:
hist_bins = torch.histc(sk_imp_mcar30[:,19], bins=3)
print(hist_bins)

tensor([255.,  33., 712.])


In [18]:
import torch

def bucketize_columns(data, bucket_specs):
    """
    Bucketizes specific columns in a dataset using different binning strategies.

    :param data: PyTorch tensor (N, D) (continuous values)
    :param bucket_specs: Dictionary {column_index: (method, num_bins or bin_edges)}
                         Example: {0: ('uniform', 3), 1: ('quantile', 4)}
    :return: Discretized tensor
    """
    data_buc = data.detach().clone()  # Ensure no gradients

    for col, bins in bucket_specs.items():
        feature = data_buc[:, col]  # Extract column
        #print(feature)

        
        min_val, max_val = feature.min(), feature.max()
        bin_edges = torch.linspace(min_val, max_val, bins + 1)
        print(bin_edges)
        bin_edges[-1] += 1e-6 
        
        # Apply bucketization
        data_buc[:, col] = torch.bucketize(feature, bin_edges,right=True) # Start bins from 0
        print(data_buc[:,col].long().unique())

    return data_buc.long()  # Convert to integer values

def compute_probabilities_torch(data, columns):
    """
    Compute probability distributions for given feature columns using PyTorch.

    :param data: PyTorch tensor of shape (N, D) (discretized)
    :param columns: List of column indices to compute probabilities
    :return: Unique values and probability tensor
    """
    unique_vals, counts = torch.unique(data[:, columns], dim=0, return_counts=True)
    probs = counts.float() / data.shape[0]
    return unique_vals, probs

def conditional_mutual_information_torch(data, X_cols, Y_cols, Z_cols, bucket_specs, delta=1):
    """
    Compute Conditional Mutual Information I(X;Y|Z) with bucketization inside.

    :param data: PyTorch tensor (N, D) (continuous values)
    :param X_cols: List of column indices for X
    :param Y_cols: List of column indices for Y
    :param Z_cols: List of column indices for Z
    :param bucket_specs: Dictionary specifying bucketization for each column
    :param delta: Smoothing parameter to avoid log(0)
    :return: Conditional Mutual Information (scalar)
    """
    # Apply bucketization inside CMI function
    bucketized_data = bucketize_columns(data, bucket_specs)

    # Compute probability distributions
    unique_Z, P_Z = compute_probabilities_torch(bucketized_data, Z_cols)
    unique_XZ, P_XZ = compute_probabilities_torch(bucketized_data, X_cols + Z_cols)
    unique_YZ, P_YZ = compute_probabilities_torch(bucketized_data, Y_cols + Z_cols)
    unique_XYZ, P_XYZ = compute_probabilities_torch(bucketized_data, X_cols + Y_cols + Z_cols)

    cmi = 0
    for i in range(len(unique_XYZ)):
        xyz = unique_XYZ[i]
        z = xyz[len(X_cols + Y_cols):]  # Extract Z values
        xz = torch.cat((xyz[:len(X_cols)], z))  # Concatenate X and Z
        yz = torch.cat((xyz[len(X_cols):len(X_cols + Y_cols)], z))  # Concatenate Y and Z

        mask_z = (unique_Z == z).all(dim=1)
        mask_xz = (unique_XZ == xz).all(dim=1)
        mask_yz = (unique_YZ == yz).all(dim=1)

        if mask_z.any() and mask_xz.any() and mask_yz.any():
            P_z = P_Z[mask_z].sum()
            P_xyz = P_XYZ[i]
            P_xz = P_XZ[mask_xz].sum()
            P_yz = P_YZ[mask_yz].sum()

            # Compute CMI (avoid log(0) by adding a small constant)
            cmi += delta * P_xyz * torch.log2((P_z * P_xyz) / (P_xz * P_yz + 1e-10))

    return cmi.item()


In [21]:
# Simulated continuous dataset (1000 samples, 7 features)
data_torch = sk_imp_mcar30 # Scale values to 0-10 range

# Define different bucketization strategies for each feature
bucket_specs = {
     
    11: 4,   # Column 11 → age (4 bins)
    19: 2,  # Column 19 → sex (2 bins)
    21: 2,  # Column 21 → label  (2 bins)
    5: 5,   # Column 5 → savings-account (5 bins)
    15: 4   # Column 15 → jobs (4 bins)
}

# Define multiple attributes for X, Y, Z
X_cols = [11,19]  # Bucketized sensitive attributes (e.g., sex, race, age)
Y_cols = [21]     # Bucketized outcome-related attributes
Z_cols = [5, 15]     # Bucketized control variables

# Compute CMI with integrated bucketization
cmi_value = conditional_mutual_information_torch(data_torch, X_cols, Y_cols, Z_cols, bucket_specs)
print(f"Conditional Mutual Information (CMI): {cmi_value:.4f}")


tensor([-1.4587, -0.2349,  0.9888,  2.2126,  3.4363])
tensor([1, 2, 3, 4])
tensor([-1.4670, -0.3927,  0.6816])
tensor([1, 2])
tensor([-1.5275, -0.4364,  0.6547])
tensor([1, 2])
tensor([-1.9608, -1.2259, -0.4911,  0.2438,  0.9786,  1.7135])
tensor([1, 2, 3, 4, 5])
tensor([-0.7001,  0.2207,  1.1416,  2.0624,  2.9832])
tensor([1, 2, 3, 4])
Conditional Mutual Information (CMI): 0.0787


In [23]:
test_german_miss_mcar30 = CMI()
cmi_german_miss_mcar30 = CMI.conditional_mutual_information(data_torch, X_cols, Y_cols, Z_cols, bucket_specs)
print(cmi_german_miss_mcar30)

0.07870997488498688
